In [1]:
import sqlite3

# Connect to the SQLite database
database_path = r'C:\Users\rishi\OneDrive\Desktop\IS\modules\Fantasy-Cricket-Game-master\Fantasy-Cricket-Game-master\cricket.db'
connection = sqlite3.connect(database_path)

# Create a cursor object
game_cursor = connection.cursor()

q = 'SELECT Player, Scored, Faced, Fours, Sixes, Bowled, Maiden, Given, Wkts, Catches, Stumping, RO FROM Match, Teams, '
queries = [
    q + 'Batsmen WHERE Match.Player = Batsmen.Batsman AND Teams.Name = Batsmen.Name AND Match.MatchID = ? AND Teams.Name = ?',
    q + 'Bowlers WHERE Match.Player = Bowlers.Bowler AND Teams.Name = Bowlers.Name AND Match.MatchID = ? AND Teams.Name = ?',
    q + 'AllRounders WHERE Match.Player = AllRounders.AR AND Teams.Name = AllRounders.Name AND Match.MatchID = ? AND Teams.Name = ?',
    q + 'WicketKeeper WHERE Match.Player = WicketKeeper.WK AND Teams.Name = WicketKeeper.Name AND Match.MatchID = ? AND Teams.Name = ?'
]
players = {}
match, team = "Match1", "ishi"

for query in queries:
    game_cursor.execute(query, (match, team))
    result = game_cursor.fetchall()
    for res in result:
        player_name = res[0]
        performance_data = res[1:]
        players[player_name] = performance_data

print(players)


{'Kohli': (102, 98, 8, 2, 0, 0, 0, 0, 0, 0, 1), 'Rahane': (49, 75, 3, 0, 0, 0, 0, 0, 1, 0, 0), 'Dhoni': (56, 45, 3, 1, 0, 0, 0, 0, 3, 2, 0), 'Rohit': (46, 65, 5, 1, 0, 0, 0, 0, 1, 0, 0), 'Axar': (8, 4, 2, 0, 48, 2, 35, 1, 0, 0, 0), 'Pandya': (42, 36, 3, 3, 30, 0, 25, 0, 1, 0, 0), 'Ashwin': (23, 42, 3, 0, 60, 2, 45, 6, 0, 0, 0), 'Dhawan': (32, 35, 4, 0, 0, 0, 0, 0, 0, 0, 0), 'Umesh': (0, 0, 0, 0, 54, 0, 50, 4, 1, 0, 0), 'Kartick': (29, 42, 3, 0, 0, 0, 0, 0, 2, 0, 1), 'Bhuwanashwar': (15, 12, 2, 0, 60, 1, 46, 2, 0, 0, 0)}


In [122]:
import pandas as pd 
import numpy as np 


In [129]:
data = pd.read_csv(r'C:\Users\rishi\Downloads\deliveries.csv\deliveries.csv')



In [130]:
columns_to_remove = ['non_striker', 'byes',	'legbyes' ,'penalty','player_dismissed','other_wicket_type','other_player_dismissed','venue','season','start_date']
data = data.drop(columns=columns_to_remove)
data['over'] = data['ball'].apply(lambda x: int(x)+1)


In [131]:

data = data[(data['batting_team'] == 'India') | (data['bowling_team'] == 'India')]


In [132]:
data

,match_id,innings,ball,batting_team,bowling_team,striker,bowler,runs_off_bat,extras,wides,noballs,wicket_type,over
2117,5,1,0.1,Australia,India,DA Warner,JJ Bumrah,0,0,NaN,NaN,NaN,1
2118,5,1,0.2,Australia,India,DA Warner,JJ Bumrah,1,0,NaN,NaN,NaN,1
2119,5,1,0.3,Australia,India,MR Marsh,JJ Bumrah,0,0,NaN,NaN,NaN,1
2120,5,1,0.4,Australia,India,MR Marsh,JJ Bumrah,0,0,NaN,NaN,NaN,1
2121,5,1,0.5,Australia,India,MR Marsh,JJ Bumrah,0,0,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26114,45,2,47.1,Netherlands,India,PA van Meekeren,RG Sharma,0,0,NaN,NaN,NaN,48
26115,45,2,47.2,Netherlands,India,PA van Meekeren,RG Sharma,1,0,NaN,NaN,NaN,48
26116,45,2,47.3,Netherlands,India,AT Nidamanuru,RG Sharma,0,0,NaN,NaN,NaN,48
26117,45,2,47.4,Netherlands,India,AT Nidamanuru,RG Sharma,6,0,NaN,NaN,NaN,48


In [133]:
# Adjusted function to calculate batting statistics using 'batsman' column
def calculate_batting_stats(df, player_name):
    player_data = df[df['striker'] == player_name]
    deliveries_faced = len(player_data)
    total_runs = player_data['runs_off_bat'].sum() - player_data['extras'].sum()  # Exclude extras
    sixes = ((player_data['runs_off_bat'] == 6)).sum() 
    fours = (player_data['runs_off_bat'] == 4).sum() 
    
    return deliveries_faced, total_runs, sixes, fours

# Adjusted function to calculate bowling statistics
def calculate_bowling_stats(df, player_name):
    player_data = df[df['bowler'] == player_name]
    balls_bowled = len(player_data)
    maiden_overs = (player_data.groupby('over')['runs_off_bat'].sum() == 0).sum()
    runs_given = player_data['runs_off_bat'].sum()
    wickets = player_data['wicket_type'].notna().sum()
    
    return balls_bowled, maiden_overs, runs_given, wickets

# # Adjusted function to calculate fielding statistics
# def calculate_fielding_stats(df, player_name):
#     catches = df[(df['fielder'] == player_name) & (df['wicket_type'] == 'caught')].shape[0] if 'fielder' in df.columns else 0
#     stumpings = df[(df['fielder'] == player_name) & (df['wicket_type'] == 'stumped')].shape[0] if 'fielder' in df.columns else 0
#     runouts = df[(df['fielder'] == player_name) & (df['wicket_type'] == 'run out')].shape[0] if 'fielder' in df.columns else 0
    
#     return catches, stumpings, runouts

# Initialize the summary dataframe
summary_df = pd.DataFrame(columns=[
    'Match ID', 'Player Name', 'Deliveries Faced', 'Total Runs Scored', 'Sixes', 'Fours', 
    'Balls Bowled', 'Maiden Overs', 'Runs Given', 'Wickets', 'Catches', 'Stumpings', 'Run Outs'
])

# Get unique matches and players
country_bat = data[data['batting_team']=='India']
country_bowl = data[data['bowling_team']=='India']
matches = data['match_id'].unique()
players = pd.concat([country_bat['striker'], country_bowl['bowler']]).unique()

# Calculate statistics for each player in each match
for match in matches:
    match_data = data[data['match_id'] == match]
    for player in players:
        bat_data =match_data[match_data['batting_team']=='India']
        deliveries_faced, total_runs, sixes, fours = calculate_batting_stats(bat_data, player)
        bowl_data =match_data[match_data['bowling_team']=='India']
        balls_bowled, maiden_overs, runs_given, wickets = calculate_bowling_stats(bowl_data, player)
        # catches, stumpings, runouts = calculate_fielding_stats(match_data, player)
        catches,stumpings,runouts =0,0,0
        new_data = pd.DataFrame([{
            'Match ID': match,
            'Player Name': player,
            'Deliveries Faced': deliveries_faced,
            'Total Runs Scored': total_runs,
            'Sixes': sixes,
            'Fours': fours,
            'Balls Bowled': balls_bowled,
            'Maiden Overs': maiden_overs,
            'Runs Given': runs_given,
            'Wickets': wickets,
            'Catches': catches,
            'Stumpings': stumpings,
            'Run Outs': runouts
        }])
        summary_df = pd.concat([summary_df, new_data], ignore_index=True)



In [135]:
#print(summary_df)

    Match ID     Player Name Deliveries Faced Total Runs Scored Sixes Fours  \
0          5       RG Sharma                7                -2     0     0   
1          5    Ishan Kishan                1                 0     0     0   
2          5         V Kohli              119                81     0     6   
3          5         SS Iyer                3                 0     0     0   
4          5        KL Rahul              116                95     2     8   
..       ...             ...              ...               ...   ...   ...   
160       45       JJ Bumrah                0                 0     0     0   
161       45   Kuldeep Yadav                0                 0     0     0   
162       45  Mohammed Siraj                0                 0     0     0   
163       45        R Ashwin                0                 0     0     0   
164       45       SN Thakur                0                 0     0     0   

    Balls Bowled Maiden Overs Runs Given Wickets Ca

In [136]:
try:
    # Connect to SQLite database
    conn = sqlite3.connect('cricketdata.db')
    cursor = conn.cursor()

    # Create table if it doesn't exist
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS All Matches (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        match_id TEXT,
        player_name TEXT,
        deliveries_faced INTEGER,
        total_runs_scored INTEGER,
        sixes INTEGER,
        fours INTEGER,
        balls_bowled INTEGER,
        maiden_overs INTEGER,
        runs_given INTEGER,
        wickets INTEGER,
        catches INTEGER,
        stumpings INTEGER,
        run_outs INTEGER
    );
    ''')

    # Iterate over DataFrame rows and insert into database
    for index, row in summary_df.iterrows():
        cursor.execute('''
        INSERT INTO stats (
            match_id, player_name, deliveries_faced, total_runs_scored, sixes, fours,
            balls_bowled, maiden_overs, runs_given, wickets, catches, stumpings, run_outs
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            row['Match ID'], row['Player Name'], row['Deliveries Faced'], row['Total Runs Scored'],
            row['Sixes'], row['Fours'], row['Balls Bowled'], row['Maiden Overs'],
            row['Runs Given'], row['Wickets'], row['Catches'], row['Stumpings'], row['Run Outs']
        ))

    # Commit changes
    conn.commit()
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Ensure the connection is closed
    if conn:
        conn.close()


array(['RG Sharma', 'Ishan Kishan', 'V Kohli', 'SS Iyer', 'KL Rahul',
       'HH Pandya', 'Shubman Gill', 'SA Yadav', 'RA Jadeja',
       'Mohammed Shami', 'JJ Bumrah', 'Kuldeep Yadav', 'Mohammed Siraj',
       'R Ashwin', 'SN Thakur'], dtype=object)

In [153]:
# Create table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Stats  (
    player_name TEXT,
    Category TEXT,
    total_matches INTEGER,
    total_runs_scored INTEGER,
    hundred_s INTEGER,
    fifty_s INTEGER,
    wickets INTEGER,
    Value INTEGER
    );
    ''')

In [139]:
# Connect to SQLite database
conn = sqlite3.connect('cricketdata.db')
cursor = conn.cursor()# Rename the table
cursor.execute('ALTER TABLE stats RENAME TO "All Matches";')

# Commit changes and close the connection
conn.commit()

In [151]:
# SQL query to insert player names
quer = 'INSERT INTO Stats (player_name, Category, total_matches, total_runs_scored, hundred_s, fifty_s, wickets, Value) VALUES (?, ?, ?, ?, ?, ?, ?, ?)'

# Example data for each player
# You will need to provide actual values for the rest of the fields


In [154]:
# List of player data with actual values
players_data = [
    ('RG Sharma', 'BAT', 250, 8500, 30, 50, 0, 150),
    ('Ishan Kishan', 'BAT', 55, 1800, 3, 12, 0, 60),
    ('V Kohli', 'BAT', 300, 10500, 40, 60, 0, 200),
    ('SS Iyer', 'BAT', 120, 3500, 7, 20, 0, 90),
    ('KL Rahul', 'WK', 150, 5000, 10, 25, 0, 120),
    ('HH Pandya', 'AR', 100, 2500, 2, 15, 70, 100),
    ('Shubman Gill', 'BAT', 70, 2500, 6, 20, 0, 70),
    ('SA Yadav', 'AR', 90, 3000, 1, 18, 80, 85),
    ('RA Jadeja', 'AR', 250, 3200, 0, 15, 250, 160),
    ('Mohammed Shami', 'BOWL', 150, 0, 0, 0, 200, 90),
    ('JJ Bumrah', 'BOWL', 120, 0, 0, 0, 180, 110),
    ('Kuldeep Yadav', 'BOWL', 85, 0, 0, 0, 120, 70),
    ('Mohammed Siraj', 'BOWL', 60, 0, 0, 0, 100, 65),
    ('R Ashwin', 'BOWL', 170, 0, 0, 0, 400, 130),
    ('SN Thakur', 'WK', 50, 0, 0, 0, 0, 50)
]

# SQL query to insert player data
quer = '''
INSERT INTO Stats (
    player_name, Category, total_matches, total_runs_scored, hundred_s, fifty_s, wickets, Value
) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
'''

# Insert data into the Stats table
cursor.executemany(quer, players_data)

# Commit changes and close the connection
conn.commit()

In [155]:
# Create 'all_games' table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS all_games (
        match_number INTEGER PRIMARY KEY,
        match_detail TEXT
    )
''')

# List of match details
matches_data = [
    (5, 'India vs Australia'),
    (9, 'India vs Afghanistan'),
    (12, 'India vs Pakistan'),
    (17, 'India vs Bangladesh'),
    (21, 'India vs New Zealand'),
    (29, 'India vs England'),
    (33, 'India vs Sri Lanka'),
    (37, 'India vs South Africa'),
    (45, 'India vs Netherlands')
]

# SQL query to insert match data
quer = '''
INSERT INTO all_games (match_number, match_detail) VALUES (?, ?)
'''

# Insert data into the all_games table
cursor.executemany(quer, matches_data)

# Commit changes and close the connection
conn.commit()

In [156]:
# Create 'all_rounder' table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS all_rounder (
        team_name TEXT,
        Player_name TEXT,
        FOREIGN KEY(team_name) REFERENCES teams(team_name),
        FOREIGN KEY(Player_name) REFERENCES Stats(player_name)
    )
''')

# Create 'batsmen' table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS batsmen (
        team_name TEXT,
        player_name,
        FOREIGN KEY(team_name) REFERENCES teams(team_name),
        FOREIGN KEY(player_name) REFERENCES Stats(player_name)
    )
''')

# Create 'bowlers' table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS bowlers (
        team_name TEXT,
        player_name TEXT,
        FOREIGN KEY(team_name) REFERENCES teams(team_name),
        FOREIGN KEY(player_name) REFERENCES Stats(player_name)
    )
''')

# Create 'wicket_keeper' table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS wicket_keeper (
        team_name TEXT,
        player_name TEXT,
        FOREIGN KEY(team_name) REFERENCES teams(team_name),
        FOREIGN KEY(player_name) REFERENCES Stats(player_name)
    )
''')

# Commit changes and close the connection
conn.commit()


In [2]:
def getQueryResult( query):
    try:
        result=[]
        for q in query:
            connection.execute(q)
            result.append(game_cursor.fetchall())
        return result
    except Exception as e:
        print ('from method getQueryResult')